In [ ]:
# ==============================================================================
# PARTE 1: CONFIGURAÇÃO E CARREGAMENTO DE DADOS
# ==============================================================================
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
import warnings

# Ignorar avisos futuros para manter a saída limpa
warnings.simplefilter(action='ignore', category=FutureWarning)

print("--- Módulo de Análise Laboratorial Inteligente ---")
print("Passo 1: Carregando as bases de conhecimento...")

# Função para converter o link do Google Sheets para o formato de download CSV
def formatar_url_gsheets(url):
    return url.replace('/edit?usp=sharing', '/export?format=csv')

# URLs das suas planilhas
#url_triador = formatar_url_gsheets('https://docs.google.com/spreadsheets/d/1WGxbB1jQVfC7Lfd7dyvMTymV6Q8AAkUuQ7J_gyNG53Y/edit?usp=sharing')
url_case1 = formatar_url_gsheets('https://docs.google.com/spreadsheets/d/1Js5wuduycKnQ8A2zh1G9wOCpAw3q15O_crbRuD3Wpl0/edit?usp=sharing')
url_case2 = formatar_url_gsheets('https://docs.google.com/spreadsheets/d/1bx61oNSc9Rb8-5GGenC8XeTmeRwqp91E_F3uFdNvaPI/edit?usp=sharing')
url_case3 = formatar_url_gsheets('https://docs.google.com/spreadsheets/d/15kRKMAuCe27L0qyPKO10S5WS6G0bEmpU0vLtaCDxOL4/edit?usp=sharing')

# Carregando os dados em DataFrames
try:
    #df_triador = pd.read_csv(url_triador)
    df_case1 = pd.read_csv(url_case1)
    df_case2 = pd.read_csv(url_case2)
    df_case3 = pd.read_csv(url_case3)
    print("Sucesso! Todas as 4 tabelas foram carregadas.\n")
except Exception as e:
    print(f"Erro ao carregar as planilhas: {e}")
    print("Verifique se os links estão corretos e compartilhados como 'Qualquer pessoa com o link'.")

# ==============================================================================
# PARTE 2: PRÉ-PROCESSAMENTO E TREINAMENTO DOS MODELOS ESPECIALISTAS
# ==============================================================================

print("Passo 2: Treinando os modelos especialistas...")

def treinar_modelo_especialista(df, nome_coluna_diagnostico):
    """
    Função reutilizável para pré-processar e treinar um modelo de árvore de decisão.
    """
    # Tratar células vazias como 'Nao_Informado'
    df_tratado = df.fillna('Nao_Informado')

    # Separar as features (X) do label (y)
    X = df_tratado.drop(nome_coluna_diagnostico, axis=1)
    y = df_tratado[nome_coluna_diagnostico]

    # Converter colunas de texto em colunas numéricas (One-Hot Encoding)
    X_encoded = pd.get_dummies(X)

    # Treinar o modelo
    modelo = DecisionTreeClassifier(random_state=42)
    modelo.fit(X_encoded, y)

    # Retornar o modelo treinado e as colunas usadas no treino (essencial!)
    return modelo, X_encoded.columns

# Treinando um modelo para cada Case
modelo_case1, colunas_case1 = treinar_modelo_especialista(df_case1, 'DIAGNÓSTICO_VERMELHA')
print("- Modelo Especialista do Case 1 (Série Vermelha) treinado.")

modelo_case2, colunas_case2 = treinar_modelo_especialista(df_case2, 'DIAGNÓSTICO_BRANCA')
print("- Modelo Especialista do Case 2 (Série Branca) treinado.")

modelo_case3, colunas_case3 = treinar_modelo_especialista(df_case3, 'DIAGNÓSTICO_PLAQUETAS')
print("- Modelo Especialista do Case 3 (Plaquetas) treinado.\n")

# Agrupando os modelos e colunas para fácil acesso
modelos_e_colunas = {
    'Case 1': (modelo_case1, colunas_case1),
    'Case 2': (modelo_case2, colunas_case2),
    'Case 3': (modelo_case3, colunas_case3)
}

# ==============================================================================
# PARTE 3: O ORQUESTRADOR - A FUNÇÃO PRINCIPAL DE ANÁLISE
# ==============================================================================

print("Passo 3: Construindo o orquestrador do sistema...")


def analisar_hemograma_completo(dados_paciente, modelos):

    relatorio_final = {}

    # --- NÍVEL 2: MÓDULOS DE ANÁLISE (acionando os modelos) ---
    print("Nível 2 (Especialistas): Acionando modelos...")

    for case in modelos:
        modelo, colunas_treino = modelos[case]

        # Preparar os dados do paciente para o modelo específico
        df_paciente = pd.DataFrame([dados_paciente])
        df_paciente_encoded = pd.get_dummies(df_paciente)

        # Garantir que o df do paciente tenha exatamente as mesmas colunas do treino
        df_paciente_final = df_paciente_encoded.reindex(columns=colunas_treino, fill_value=0)

        # Fazer a predição
        predicao = modelo.predict(df_paciente_final)

        # Adicionar ao relatório
        nome_analise = f"Analise_{case.replace(' ', '_')}"
        relatorio_final[nome_analise] = predicao[0]

    return relatorio_final

print("Sucesso! Sistema pronto para receber exames.\n")


# ==============================================================================
# PARTE 4: EXEMPLO DE USO
# ==============================================================================

# Vamos simular um novo paciente com anemia e plaquetas baixas.
# Note que não informamos os parâmetros opcionais (Eosinofilos, Monocitos, etc.)

print ('********** INSERIR NAS PERGUNTAS A SEGUIR: ALTO, NORMAL OU BAIXO; SIM OU NÃO *********\n OBS: EM PARAMETROS AUSENTES, DEIXAR EM BRANCO\n')
Eritrocitos = input('Eritrocitos: ').lower()
Hematocrito = input('Hematocrito: ').lower()
VCM = input('VCM: ').lower()
CHCM = input('CHCM: ').lower()
RDW = input('RDW: ').lower()
Reticulocitos = input('Reticulocitos: ').lower()
Leucocitos_Totais = input('Leucocitos: ').lower()
Neutrofilos_Seg = input('Neutrofilos Segmentados: ').lower()
Neutrofilos_Bast = input('Neutrofilos Bastonados: ').lower()
Linfocitos = input('Linfocitos: ').lower()
Monocitos = input('Monocitos: ').lower()
Eosinofilos = input('Eosinofilos: ').lower()
Plaquetas = input('Plaquetas: ').lower()
PDW = input('PDW: ').lower()
VPM = input('VPM: ').lower()
Observacao_Agregados = input('Observação Agregados: ').lower()
Grafico_Plq = input('Gráfico Plq: ').lower()


novo_paciente = {
    'Eritrocitos': Eritrocitos,
    'Hematocrito': Hematocrito,
    'VCM': VCM,
    'CHCM': CHCM,
    'RDW': RDW,
    'Reticulocitos': Reticulocitos,
    'Leucocitos_Totais': Leucocitos_Totais,
    'Neutrofilos_Seg': Neutrofilos_Seg,
    'Neutrofilos_Bast': Neutrofilos_Bast,
    'Linfocitos': Linfocitos,
    'Monocitos': Monocitos,
    'Eosinofilos': Eosinofilos,
    'Plaquetas': Plaquetas,
    'PDW': PDW,
    'VPM': VPM,
    'Observacao_Agregados': Observacao_Agregados,
    'Gráfico Plq': Grafico_Plq
  }

# Executa a análise completa
laudo_final = analisar_hemograma_completo(novo_paciente, modelos_e_colunas)

# Imprime o laudo final de forma organizada
print("\n================== LAUDO INTELIGENTE GERADO ==================")
if laudo_final:
    for chave, valor in laudo_final.items():
        print(f"-> {chave}: {valor}")
else:
    print("Não foi possível gerar um laudo.")
print("============================================================")

--- Módulo de Análise Laboratorial Inteligente ---
Passo 1: Carregando as bases de conhecimento...
Sucesso! Todas as 4 tabelas foram carregadas.

Passo 2: Treinando os modelos especialistas...
- Modelo Especialista do Case 1 (Série Vermelha) treinado.
- Modelo Especialista do Case 2 (Série Branca) treinado.
- Modelo Especialista do Case 3 (Plaquetas) treinado.

Passo 3: Construindo o orquestrador do sistema...
Sucesso! Sistema pronto para receber exames.

********** INSERIR NAS PERGUNTAS A SEGUIR: ALTO, NORMAL OU BAIXO; SIM OU NÃO *********
 OBS: EM PARAMETROS AUSENTES, DEIXAR EM BRANCO

Eritrocitos: Normal
Hematocrito: Normal
VCM: Alto
CHCM: Normal
RDW: Alto
Reticulocitos: 
Leucocitos: Normal
Neutrofilos Segmentados: Alto
Neutrofilos Bastonados: Normal
Linfocitos: Baixo
Monocitos: Alto
Eosinofilos: Normal
Plaquetas: Normal
PDW: Alto
VPM: Alto
Observação Agregados: Não
Gráfico Plq: Alto
Nível 2 (Especialistas): Acionando modelos...

================== LAUDO INTELIGENTE GERADO =========